In [42]:
# Install the dependencies

In [43]:
pip install librosa soundfile pyaudio

In [44]:
# Make the necessary imports

In [10]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
# Feature Extraction ( Check sample data processing folder to know the concepts )

In [12]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])
        if chroma:
            stft = np.abs(librosa.stft(X))
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel_spec = librosa.feature.melspectrogram(y=X, sr=sample_rate)
            mel = np.mean(mel_spec.T, axis=0)
            result = np.hstack((result, mel))
    return result


In [13]:
# Define dictionary for the emotions in the RAVDESS dataset

In [56]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral', 'happy', 'sad', 'angry','fearful', 'disgust']

In [57]:
# Load the data and extract features for each sound file

In [58]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"E:\Downloads\speech-emotion-recognition-ravdess-data\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9) 

In [59]:
# Split the dataset

In [60]:
x_train,x_test,y_train,y_test = load_data(test_size=0.21)

In [61]:
# No. of samples in training and testing set

In [62]:
print((x_train.shape[0], x_test.shape[0]))

(834, 222)


In [63]:
# No. of features extracted

In [64]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [65]:
# Initialize the Multi Layer Perceptron Classifier

In [201]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [202]:
# Train the model

In [203]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [204]:
# Predict for the test set

In [205]:
y_pred=model.predict(x_test)

In [206]:
# Calculate the accuracy of the model

In [207]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.02%


In [208]:
# Function to predict emotion from custom audio

In [209]:
def predict_emotion_from_audio(file_path):
    # Extract features from the custom audio
    features = extract_feature(file_path, mfcc=True, chroma=True, mel=True)

    # Reshape the features for prediction (make it 2D array: 1 sample, n_features)
    features = features.reshape(1, -1)

    # Predict using the trained model
    emotion = model.predict(features)

    return emotion[0]

In [210]:
audio_file = r"E:\Downloads\speech-emotion-recognition-ravdess-data\Actor_24\03-01-07-01-02-02-24.wav"

In [211]:
predicted_emotion = predict_emotion_from_audio(audio_file)
print(f"Predicted Emotion: {predicted_emotion}")

Predicted Emotion: disgust
